In [3]:
import os
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re
from os.path import exists

In [ ]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth',1000)

<br>(?!) - negative lookahead
<br>(?=) - positive lookahead
<br>(?<=) - positive lookbehind
<br>(?<!) - negative lookbehind
<br>(?>) - atomic group

In [14]:
def extract_att(df_product,direction):
    df_product['temp_title']=df_product[direction+'title'].fillna('').str.lower()
    df_product['temp_brand_name']=df_product['brand_name'].fillna('').str.lower()
    df_product['temp_brand_name']=df_product['temp_brand_name'].str.replace(r'\+|\?|\|','')    
    df_product[direction+'model']=df_product.apply(lambda row : re.search(r'(?<=('+row['temp_brand_name']+'\s))[A-Za-z]+\s?[A-Za-z\+\.\d]*\s?[A-Za-z\+\.\d]*\s?|$',row['temp_title']).group(), axis=1)
    df_product[direction+'cp_model']=df_product.apply(lambda row : re.search(r'(?<=('+'for'+'\s))[A-Za-z]+\s?[A-Za-z\+\.\d]*\s?[A-Za-z\+\.\d]*\s?|$',row['temp_title']).group(), axis=1)

    bi_name_regex={'is_watch':r'[wW]atch'
                   ,'is_apple_watch':r'[Aa]pple\s?[Ww]atch'
                   ,'is_band':r'[bB]and'
                   ,'is_screen_protector':r'[pP]rotector|shield|guard'
                   ,'is_case':r'[cC]ase'
                   ,'is_cable':r'[cC]able'
                   ,'is_charger':r'charg[er|ing]|powerbank'
                   ,'is_stand':r'(stand)|(mount)|(holder)|(arm)|clamp'
                   ,'is_fan':r'[fF]an'
                   ,'is_magnifier':r'screen\smagnifier'
                   ,'is_adapter':r'adapter'
                  }


    att_name_regex={
        'size_mm':r'([-.0-9]+mm)'
        ,'size_inch':r'([\.0-9]+\s?(inch|")+)'
        ,'size_inch_min':r'(((?<!(\-))|(?<!(\to)))\s?([0-9]+\.?[0-9]*\s?((?=(\'\'))|(?=(inches))|(?=(inch))|(?=(")))))'
        ,'size_inch_max':r'(((?<=(\-))|(?<=(\to)))\s?([0-9]+\.?[0-9]*\s?((?=(\'\'))|(?=(inches))|(?=(inch))|(?=(")))))'
        ,'storage':r'([2|4|8|16|64|128]gb/s?[Rr][Aa][Mm])'
    }
    
    #is type attributes
    for name in bi_name_regex:
        regex_t=bi_name_regex[name]
        df_product[direction+name]=df_product['temp_title'].str.contains(regex_t)

    #attribtes
    for name in att_name_regex:
        regex_t=att_name_regex[name]
        df_product[direction+name]=df_product['temp_title'].str.extract(regex_t)[0]
    
    df_product['is_accessory']=np.where(df_product.is_band|df_product.is_screen_protector|df_product.is_case
                                        |df_product.is_cable|df_product.is_charger|df_product.is_stand
                                        |df_product.is_fan|df_product.is_magnifier|df_product.is_adapter,True,False)
        #fix data type
    df_product['size_inch_max']=pd.to_numeric(df_product.size_inch_max.str.strip())
    df_product['size_inch_min']=pd.to_numeric(df_product.size_inch_min.str.strip())
    
    #drop prefix
    df_product=df_product.loc[:,~df_product.columns.str.startswith('temp')]
    
    return df_product